# Delete a Session

This notebook deletes a session record from all related tables by session_id and updates Summary_Sessions.

**Note:** 
- You only need to provide the `session_id` (which is a hash of user_id and boyfriend_name)
- The `session_id` value is stored in the `id` column in all database tables
- You can get the `id` value from the `list_sessions()` function output (see `06_list_sessions.ipynb`)


In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Add project root to path
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))

from src.adapters.database.database_handler import DatabaseHandler
from src.utils.summary_updater import update_summary_after_delete


## Configuration

Set `USE_DYNAMODB = True` to use DynamoDB, or `False` to use CSV files.


In [ ]:
# Configuration
USE_DYNAMODB = True  # Set to False to use CSV instead


In [ ]:
def delete_session(session_id, db_write_allowed: bool = True) -> bool:
    """
    Delete a session record from all related tables based on session_id.
    Also updates Summary_Sessions table.
    
    Args:
        session_id: The session ID value (stored in the 'id' column in database tables).
                    This is a hash of user_id and boyfriend_name.
                    Can be int or str (will be converted to int for comparison).
        db_write_allowed: If True, use DynamoDB; if False, use CSV
        
    Returns:
        True if deletion was successful, False otherwise
        
    Note:
        The session_id parameter corresponds to the 'id' column in all tables.
        The 'id' column is stored as a number (int) in the database.
        You can get this value from list_sessions() output.
    """
    # Convert session_id to int to ensure type match with database
    try:
        session_id_int = int(session_id)
    except (ValueError, TypeError):
        print(f"[ERROR] session_id must be a number (int or string representation of int). Got: {type(session_id)} = {session_id}")
        return False
    
    print("=" * 60)
    print(f"Deleting Session Record")
    print("=" * 60)
    print(f"Session ID: {session_id_int}")
    print(f"Backend: {'DynamoDB' if db_write_allowed else 'CSV'}")
    print("=" * 60)
    
    db_handler = DatabaseHandler(db_write_allowed=db_write_allowed)
    
    try:
        # Load the record to get its values before deleting
        print(f"\n[1] Loading record with session_id {session_id_int} from session_responses...")
        session_responses = db_handler.load_table("session_responses")
        
        if session_responses.empty:
            print("[ERROR] session_responses table is empty")
            db_handler.close()
            return False
        
        # Ensure 'id' column is numeric for proper comparison
        session_responses["id"] = pd.to_numeric(session_responses["id"], errors='coerce')
        
        # Find the record by session_id (session_id is stored in the 'id' column as int)
        record = session_responses[session_responses["id"] == session_id_int]
        
        if record.empty:
            print(f"[ERROR] Record with session_id={session_id_int} not found")
            db_handler.close()
            return False
        
        # Get values for summary update
        row = record.iloc[0]
        deleted_toxic_score = float(row.get("toxic_score", 0))
        deleted_filter_violations = int(row.get("filter_violations", 0))
        
        print(f"[OK] Found record:")
        print(f"     Session ID: {session_id_int}")
        print(f"     User ID: {row.get('user_id')}")
        print(f"     Boyfriend Name: {row.get('boyfriend_name')}")
        print(f"     Toxic Score: {deleted_toxic_score}")
        print(f"     Filter Violations: {deleted_filter_violations}")
        
        # Delete the record from all related tables
        print(f"\n[2] Deleting records with session_id {session_id_int} from all tables...")
        tables_to_delete = [
            "session_responses",
            "session_gtk_responses",
            "session_feedback",
            "session_toxicity_rating",
            "session_insights",
        ]
        
        deleted_count = 0
        for table_name in tables_to_delete:
            try:
                # Delete record using session_id_int (which is stored in the 'id' column as int)
                if db_handler.delete_record(table_name, session_id_int, id_column="id"):
                    deleted_count += 1
                    print(f"[OK] Deleted from {table_name}")
                else:
                    print(f"[INFO] No record found in {table_name} (may not exist for this session)")
            except Exception as e:
                print(f"[WARNING] Could not delete from {table_name}: {e}")
        
        if deleted_count == 0:
            print("[ERROR] Failed to delete any records")
            db_handler.close()
            return False
        
        # Update Summary_Sessions
        print(f"\n[3] Updating Summary_Sessions...")
        update_success = update_summary_after_delete(
            db_handler=db_handler,
            deleted_toxic_score=deleted_toxic_score,
            deleted_filter_violations=deleted_filter_violations,
        )
        
        if not update_success:
            print("[WARNING] Record deleted but Summary_Sessions update failed")
        
        db_handler.close()
        
        print("\n" + "=" * 60)
        print("[SUCCESS] Session record deleted and Summary_Sessions updated!")
        print("=" * 60)
        return True
        
    except Exception as e:
        print(f"\n[ERROR] Error deleting session: {e}")
        import traceback
        print(f"[ERROR] Traceback: {traceback.format_exc()}")
        db_handler.close()
        return False


In [ ]:
# Example: Delete a session
# Uncomment and modify the session_id below to delete a session
# Note: session_id is the value stored in the 'id' column in the database tables
# You can get this value from the list_sessions() function output (shown as "Session ID")

# delete_session(
#     session_id="12",  # This is the 'id' column value from the table
#     db_write_allowed=USE_DYNAMODB
# )
